In [244]:
import random
import re
from urllib.parse import urlparse, parse_qs, parse_qsl, quote

import pandas as pd
import requests

from tqdm import tqdm

from bs4 import BeautifulSoup
import bs4

URL_ROOT = 'https://en.wiktionary.org'

soups = {}
def get_soup(path):
    if path in soups:
        return soups[path]
    
    res = requests.get(f"{URL_ROOT}{path}")
    soup = BeautifulSoup(res.text)
    soups[path] = soup
    return soup

# e.g. get_category_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_category_pages(start_path):
    paths = [start_path]    
    next_path = start_path
    soup = get_soup(next_path)
    next_link = soup.find('a', string='next page')
    while next_link:
        next_path = next_link['href']
        paths.append(next_path)
        soup = get_soup(next_path)
        next_link = soup.find('a', string='next page')
    return paths

# e.g. get_lemma_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_lemma_pages(path):
    soup = get_soup(path)
    return [{'href': a['href'], 'title': a['title']} for a in soup.css.select('div#mw-pages li a')]

In [245]:
pd.set_option('display.max_colwidth', 250)

In [246]:
# Category:Armenian_terms_with_usage_examples
# Category:Old_Armenian_terms_with_usage_examples

lps = ([lp for cp in get_category_pages('/wiki/Category:Armenian_terms_with_usage_examples')
    for lp in get_lemma_pages(cp)])
lemmas = pd.DataFrame(lps)
lemmas

,href,title
0,/wiki/%D5%A1,ա
1,/wiki/%D5%A1%CC%88,ա̈
2,/wiki/%D5%A1%CC%8A,ա̊
3,/wiki/%D5%A1%D5%A2%D5%A5%D5%A9%D5%A1%D5%BD%D5%B8%D6%82%D5%B6%D5%AF,աբեթասունկ
4,/wiki/%D5%A1%D5%A3%D5%BC%D5%A1%D5%BE,ագռավ
...,...,...
1788,/wiki/%D5%96%D6%80%D5%B8%D6%82%D5%B6%D5%A6%D5%A5,Ֆրունզե
1789,/wiki/%D5%9B,՛
1790,/wiki/%D5%9E,՞
1791,/wiki/%D6%89,։


In [247]:
def getLanguageSections(soup):
    headings = soup.select('div#bodyContent div.mw-heading2')
    
    heading_map = {}

    for heading in headings:
        language = heading.find('h2').text
        content = []
        for sib in heading.next_siblings:
            if sib == '\n':
                continue
            if type(sib) == bs4.element.Comment:
                continue
            if sib in headings:
                break
            content.append(sib)
        heading_map[language] = content
    return heading_map

In [248]:
def getHeadingLevel(cs):
    levels = [c for c in cs if c != 'mw-heading']
    if len(levels) > 0:
        return levels[0]

def getSubSections(section):
    results = []
    result = {}
    current_header = None
    current_subsection = bs4.Tag(name='div')
    top_heading_level = None
    for tag in section:
        classes = tag.attrs.get('class', [])
        if tag.name == 'div' and 'mw-heading' in classes:
            heading_level = getHeadingLevel(classes)
            if not top_heading_level:
                top_heading_level = heading_level
            elif heading_level == top_heading_level and result:
                # the first time you encounter the top-level heading again...
                if len(result) > 1: # this means there were other headings in between
                    results.append(result)
                    result = {}
                else: # consecutive top-level headings means flat hierarchy
                    top_heading_level = '_' # non-existent heading
            if current_header:
                result[current_header] = current_subsection
            current_header = tag.text.replace('[edit]', '')
            current_subsection = bs4.Tag(name='div')
        else:
            # if not heading, add to current heading's subsection 
            current_subsection.append(tag)
    if current_header:
        result[current_header] = current_subsection
    results.append(result)
    return results

In [434]:
#entries = {}

In [435]:
for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas.iterrows()]):
#for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas[lemmas.title.isin(['աման', 'փիս', '-անք', 'համ'])].iterrows()]):
    if title in entries:
        continue
    sections = getLanguageSections(get_soup(href))
    if 'Armenian' not in sections:
        continue
    entries[title] = getSubSections(sections['Armenian'])

100%|██████████| 1793/1793 [16:53<00:00,  1.77it/s]


In [436]:
results = pd.DataFrame([dict(title=k, idx=i+1, **r) for k,v in entries.items() for i,r in enumerate(v)]).set_index(['title', 'idx'])

In [523]:
from markdownify import markdownify as md

print(results.Declension[lambda x: x.apply(lambda y: type(y) == bs4.Tag and len(y) > 0)].sample(1).apply(str).apply(md, strip=['a']).str.strip().iloc[0])

*i*-type, animate (Eastern Armenian)



|  | singular | plural |
| --- | --- | --- |
| nominative | արարած (ararac) | արարածներ (araracner) |
| dative | արարածի (araraci) | արարածների (araracneri) |
| ablative | արարածից (araracicʻ) | արարածներից (araracnericʻ) |
| instrumental | արարածով (araracov) | արարածներով (araracnerov) |
| locative | — | — |
|  | definite forms | | | |
| nominative | արարածը/արարածն (araracə/araracn) | արարածները/արարածներն (araracnerə/araracnern) |
| dative | արարածին (araracin) | արարածներին (araracnerin) |
|  | 1st person possessive forms (my) | | | |
| nominative | արարածս (araracs) | արարածներս (araracners) |
| dative | արարածիս (araracis) | արարածներիս (araracneris) |
| ablative | արարածիցս (araracicʻs) | արարածներիցս (araracnericʻs) |
| instrumental | արարածովս (araracovs) | արարածներովս (araracnerovs) |
| locative | — | — |
|  | 2nd person possessive forms (your) | | | |
| nominative | արարածդ (araracd) | արարածներդ (araracnerd) |
| dative | արարածիդ (a

In [489]:
results.loc[('գազար', 1)]

Pronunciation           
Etymology               
Declension            []
Noun                  []
Alternative forms     []
Adjective            NaN
Derived terms         []
References            []
Related terms        NaN
Verb                 NaN
Synonyms              []
Adverb               NaN
Inflection           NaN
Further reading      NaN
Usage notes          NaN
Etymology 2          NaN
Etymology 1          NaN
See also             NaN
Descendants           []
Suffix               NaN
Conjugation          NaN
Proper noun          NaN
Postposition         NaN
Pronoun              NaN
Antonyms             NaN
Particle             NaN
Interjection         NaN
Conjunction          NaN
Preposition          NaN
Etymology 3          NaN
Letter               NaN
Prefix               NaN
Determiner           NaN
Punctuation mark     NaN
Hyponyms             NaN
Numeral              NaN
Phrase               NaN
Etymology 4          NaN
Article              NaN
Name: (գազար, 1), dtype: 

In [438]:
columns_by_freq = results.count().sort_values(ascending=False)
results = results[columns_by_freq.index] #[lambda x:x['Verb'].notna()]
columns_by_freq.to_frame().style.bar()

,0
Pronunciation,1780
Etymology,1622
Declension,1430
Noun,1049
Alternative forms,520
Adjective,503
Derived terms,451
References,262
Related terms,226
Verb,224


In [439]:
def process_pronunciation(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return '\n'.join([li.text for li in tag.select('li') if not 'Audio' in li.text and not li.text == ''])

results['Pronunciation'] = results['Pronunciation'].apply(process_pronunciation)

In [440]:
def process_basic_text(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return tag.text.strip()


results['Etymology'] = results['Etymology'].apply(process_basic_text)
results['Etymology'] = results['Etymology 1'].apply(process_basic_text).where(results['Etymology 1'].notna(), results['Etymology'])
results['Etymology'] = results['Etymology 2'].apply(process_basic_text).where(results['Etymology 2'].notna(), results['Etymology'])

In [445]:
parts_of_speech = ['Verb',
 'Adverb',
 'Particle',
 'Pronoun',
 'References',
 'Suffix',
 'Proper noun',
 'Conjunction',
 'Postposition',
 'Interjection',
 'Preposition',
 'Determiner',
 'Punctuation mark',
 'Numeral',
 'Prefix',
 'Article', 
 'References',
 'Suffix',
 'Proper noun',
 'Postposition',
 'Pronoun',
 'Interjection',
 'Particle',
 'Conjunction',
 'Preposition',
 'Prefix',
 'Letter',
 'Determiner',
 'Punctuation mark',
 'Numeral']

In [450]:
results[parts_of_speech].count(axis=1)[lambda x: x>0]

title    idx
ա        1      1
         3      1
         4      4
ա̈       1      3
ա̊       1      3
               ..
Ֆրունզե  1      2
՛        1      4
՞        1      2
։        1      2
’        1      4
Length: 787, dtype: int64

In [447]:
def process_definition(tag):
    if type(tag) != bs4.Tag:
        return tag
    definitions = []
    if not tag.find('ol'):
        return [{'definition': 'ERROR!!!!'}]
    for li in tag.find('ol').find_all('li'):
        if li.find('dl'):
            detail_text = li.find('dl').text
            definitions.append({'definition': li.text.replace(detail_text, '').strip(), 'detail': li.find('dl').text})
        else:
            definitions.append({'definition': li.text})
    return definitions


In [452]:
pos = 'Verb'
results[results[pos].notna()][pos].sample(10).apply(process_definition).apply(pd.Series).stack().apply(pd.Series).fillna('')

definition  \
title    idx                                                                                             
հավակնել 1   0  to aspire to acquire something (e.g., position or status), to pretend, to lay claim to   
ալարել   1   0                    (Ararat, Tbilisi, Erzurum, Van, colloquial, intransitive) to be lazy   
թոթվել   1   0                                                                               ERROR!!!!   
կարոտել  1   0                                           to miss, yearn, long for someone or something   
բրթել    1   0                                                                     (dialectal) to push   
պատահել  1   0                                          (intransitive) to meet accidentally, bump into   
             1                                                         (intransitive) to happen, occur   
ստանձնել 1   0                                                                               ERROR!!!!   
ածել     1   0                                                                           to lay (eggs)   
             1                                                (dialectal) to pour, to strew (a liquid)   
             2                                              (dialectal) to play (a musical instrument)   
տուժող   1   0                                                    subject participle of տուժել (tužel)   
զգացվել  1   0                                          mediopassive of զգացնել (zgacʻnel): to be felt   
             1                                           (intransitive) to be (deeply) moved / touched   

                                                                                                                                                                 detail  
title    idx                                                                                                                                                             
հավակնել 1   0                                      Նա հավակնում է վարչապետի պաշտոնին։ ― Na havaknum ē varčʻapeti paštonin. ― He aspires to the post of prime minister.  
ալարել   1   0  ալարում եմ վեր կենալ ― alarum em ver kenal ― I am too lazy to get up\nով ալարի, ոչ դալարի ― ov alari, očʻ dalari ― He who is lazy won't prosper/succeed  
թոթվել   1   0                                                                                                                                                           
կարոտել  1   0                                                                                                    ես քեզ կարոտել էի ― es kʻez karotel ēi ― I missed you  
բրթել    1   0                                                  Synonyms: բոթել (botʻel), հրել (hrel), հրմշտել (hrmštel)\nմի՛ բրթի ինձ ― mí brtʻi inj ― do not push me!  
պատահել  1   0                                                                                                                                                           
             1                                                                                                    ի՞նչ է պատահել ― i?nčʻ ē patahel ― what has happened?  
ստանձնել 1   0                                                                                                                                                           
ածել     1   0                                                                                                                         ձու ածել ― ju acel ― to lay eggs  
             1                                                                                                                                                           
             2                                                                                                                                                           
տուժող   1   0                                                                                                                                                           
զգացվել  1   0 

In [380]:
def find_element(tag, element):
    if type(tag) != bs4.Tag:
        return tag
    return tag.find(element)

In [444]:
(pd.DataFrame({col: {elem: results[col].apply(lambda x:find_element(x,elem)).count()  for elem in ['ol', 'ul']} for col in results.columns}).T['ol']
 [lambda x:x>0][lambda x:x<90].sort_values(ascending=False).index.to_list()
)

['References',
 'Suffix',
 'Proper noun',
 'Postposition',
 'Pronoun',
 'Interjection',
 'Particle',
 'Conjunction',
 'Preposition',
 'Prefix',
 'Letter',
 'Determiner',
 'Punctuation mark',
 'Numeral']